<a href="https://colab.research.google.com/github/ivan-pirner/ml/blob/main/autoGluon_RG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# prompt: install autogluon

!pip install autogluon



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 17.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. 

In [1]:
import warnings
import os
from datetime import datetime

from autogluon.multimodal import MultiModalPredictor
from ray import tune
import pandas as pd


warnings.filterwarnings('ignore')

In [5]:
from autogluon.core.utils.loaders import load_zip

zip_file_url = "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/82048/8963053/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1737043542&Signature=RDsVnst8Hrz0v80UZ6ipSi4lb%2B7j2nGvT9QmL4eZ0mGAU4TUNxTbMHbpt%2BaYCsDVkYjTbiV30sdiBYRfmOiULgu%2BEzc5jc3qAFS%2FzDGMm9iKQzIF9615tMTOCsoFP8ocWp2XWRQswyKFXLNKi9zD37EUT%2Bp6cTIMVcw35d2mJXOgxSCnD%2FWYAK8mTEJzLd9MVjQcuO6U3%2FSYZSp6it58CydulZUsibSqNBH4C49E3zPCvst4nK51BPVyvwnrjsDZ20%2BwJCnKLGcYbNP6g4KIfbS2usNuLW7XLjPwK5jLXBJjIDeXpTLUhRqTcqrFZWAplBYvPToy0sKf1y1yDn6Sxw%3D%3D&response-content-disposition=attachment%3B+filename%3Dne-mo-2-summer-challenge.zip"
download_dir = "./nemo2"

load_zip.unzip(zip_file_url, unzip_dir=download_dir)





/content


In [7]:
# prompt: find all jpeg files in /content/nemo2/train and put them into a pandas dataframe. Each file has a label according to its parrent folder named birdX, where X stands for the integer label

import pandas as pd
import os

def create_dataframe_from_images(root_dir):
    image_data = []
    for label_dir in os.listdir(root_dir):
        label_path = os.path.join(root_dir, label_dir)
        if os.path.isdir(label_path):
            try:
                label = int(label_dir.replace("bird", ""))
            except ValueError:
                continue # Skip directories that don't follow the birdX naming convention
            for filename in os.listdir(label_path):
                if filename.lower().endswith(('.jpg', '.jpeg')):
                    image_path = os.path.join(label_path, filename)
                    image_data.append({'image_path': image_path, 'label': label})
    return pd.DataFrame(image_data)

# Example usage
train_data = create_dataframe_from_images('/content/nemo2/train/spectrograms')
print(train_data.head())

                                          image_path  label
0  /content/nemo2/train/spectrograms/bird3/2036.jpeg      3
1   /content/nemo2/train/spectrograms/bird3/792.jpeg      3
2   /content/nemo2/train/spectrograms/bird3/469.jpeg      3
3   /content/nemo2/train/spectrograms/bird3/980.jpeg      3
4    /content/nemo2/train/spectrograms/bird3/41.jpeg      3


In [ ]:
predictor = MultiModalPredictor(
    label='label',
    # presets="high_quality_hpo",
    )
predictor.fit(
    train_data=train_data,
    # config=config,
    # hyperparameters=hyperparameters,
    # hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
    # seed=1
    )
